In [1]:
import sys, os
sys.path.append('..')

import numpy as np
import cv2

from puzzlematch.processing import Draw, Processor
from puzzlematch.hardware import Webcam
from puzzlematch.env import Path
from puzzlematch.ui import display
from puzzlematch import utils

# Edges

In [ ]:
cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

camera = Webcam()
camera.open()

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Crop region of interest
    roi = frame[y:y+h,x:x+w]
    
    # Compute canny edges on each channel
    r_edges = Processor.canny_edges(roi, 'r')
    g_edges = Processor.canny_edges(roi, 'g')
    b_edges = Processor.canny_edges(roi, 'b')
    edges = cv2.add(cv2.add(r_edges,g_edges), b_edges)
    
    # Piece detection based on mask
    box = utils.mask_bbox(edges)
    if box is not None:
        xx,yy,ww,hh = box
        xx,yy = xx+x, yy+y
        frame = Draw.rectangle(frame, p1=(xx,yy), dims=(ww,hh), color=(0,0,255))
    
    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s)
    r_edges = Draw.text(r_edges, 'Edges on red channel', 'top-left', size=0.4)
    g_edges = Draw.text(g_edges, 'Edges on green channel', 'top-left', size=0.4)
    b_edges = Draw.text(b_edges, 'Edges on blue channel', 'top-left', size=0.4)
    edges   = Draw.text(edges, 'Edges combined', 'top-left', size=0.4)
    
    # Display
    images = [frame, r_edges, g_edges, b_edges, edges]
    key = display(images, wait=1)
    
    if key==27: break
        
camera.close()
cv2.destroyAllWindows()

# Clusters

In [ ]:
def x(y): pass
cv2.namedWindow('window2')
cv2.createTrackbar('k', 'window2', 2, 10, x)

cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

camera = Webcam()
camera.open()

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Crop region of interest
    roi = frame[y:y+h,x:x+w]
    bilateral = cv2.bilateralFilter(roi, 15, 75, 75)
    
    # Compute kmeans clusters
    k = int(cv2.getTrackbarPos('k', 'window2'))
    clusters = Processor.kmeans(bilateral, k)

    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s)
    bilateral = Draw.text(bilateral, 'Bilateral filter', 'top-left', size=0.4)
    clusters  = Draw.text(clusters, 'KMEANS', 'top-left', size=0.4)
    
    # Display
    images = [frame, bilateral, clusters]
    key = display(images, wait=1)
    if key==27: break
        
camera.close()
cv2.destroyAllWindows()

# HSV

In [12]:
# def x(y): pass
# cv2.namedWindow('window2')
# cv2.createTrackbar('k', 'window2', 2, 10, x)

cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

camera = Webcam()
camera.open()

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Pipeline
    roi = frame[y:y+h,x:x+w].copy()
    hsv = Processor.hsv(roi)
    hsv = cv2.GaussianBlur(hsv, (7,7), 1)
    hue,_,_ = cv2.split(hsv)
    mask = Processor.binarize(hue)
    mask = cv2.erode(mask, np.ones((3,3),np.uint8), iterations=1)
    masked = cv2.bitwise_or(roi, roi, mask=mask)

    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s)
    roi = Draw.text(roi, 'ROI', 'top-left', size=0.4)
    hue = Draw.text(hue, 'Hue (in HSV colorspace)', 'top-left', size=0.4)
    mask = Draw.text(mask, 'Mask', 'top-left', size=0.4)
    masked = Draw.text(masked, 'Masked ROI', 'top-left', size=0.4)
    
    # Display
    images = [frame, roi, hue, mask, masked]
    key = display(images, wait=1)
    if key==27: break
        
camera.close()
cv2.destroyAllWindows()